In [25]:
import requests
import pandas as pd
import time
import os
from config import token
import sqlite3
import warnings
import calendar
from scipy.spatial import cKDTree

In [26]:
# Path to the .csv.gz file
csv_gz_file_path = "Resources/2018.csv.gz"

# Read the compressed CSV file using Pandas
df = pd.read_csv(csv_gz_file_path, compression='gzip')

# Now the 'df' DataFrame contains the data from the compressed CSV file
df.head()

,AE000041196,20180101,TMAX,259,Unnamed: 4,Unnamed: 5,S,Unnamed: 7
0,AE000041196,20180101,TMIN,112,NaN,NaN,S,NaN
1,AE000041196,20180101,TAVG,186,H,NaN,S,NaN
2,AEM00041194,20180101,TMAX,250,NaN,NaN,S,NaN
3,AEM00041194,20180101,PRCP,0,NaN,NaN,S,NaN
4,AEM00041194,20180101,TAVG,209,H,NaN,S,NaN


In [28]:
df=df[['AE000041196','20180101','TMAX','259']]

# Rename columns
df = df.rename(columns={
    'AE000041196': 'station',
    '20180101': 'date',
    'TMAX': 'data_type',
    '259': 'value'
})

df.head()

,station,date,data_type,value
0,AE000041196,20180101,TMIN,112
1,AE000041196,20180101,TAVG,186
2,AEM00041194,20180101,TMAX,250
3,AEM00041194,20180101,PRCP,0
4,AEM00041194,20180101,TAVG,209


In [47]:
# List of values to keep
valid_values = ['TMAX', 'TMIN', 'TAVG', 'PRCP','SNOW','SNWD','AWND']

# Filter the DataFrame
filtered_df = df[df['data_type'].isin(valid_values)]
filtered_df.shape

(30208386, 4)

In [49]:
# Convert 'value' column to float, handling invalid entries as NaN
filtered_df['value'] = pd.to_numeric(filtered_df['value'], errors='coerce')

# Initialize empty lists to store the data
dates = []
stations = []
tmax_values = []
tmin_values = []
tavg_values = []
prcp_values = []
snow_values = []
snwd_values = []
awnd_values = []

# Iterate through the rows of the original DataFrame
for index, row in filtered_df.iterrows():
    date = row['date']
    station = row['station']
    datatype = row['data_type']
    value = row['value']

    if datatype == 'TMAX':
        tmax_values.append(value)
        tmin_values.append(None)
        tavg_values.append(None)
        prcp_values.append(None)
        snow_values.append(None)
        snwd_values.append(None)
        awnd_values.append(None)
    elif datatype == 'TMIN':
        tmax_values.append(None)
        tmin_values.append(value)
        tavg_values.append(None)
        prcp_values.append(None)
        snow_values.append(None)
        snwd_values.append(None)
        awnd_values.append(None)
    elif datatype == 'TAVG':
        tmax_values.append(None)
        tmin_values.append(None)
        tavg_values.append(value)
        prcp_values.append(None)
        snow_values.append(None)
        snwd_values.append(None)
        awnd_values.append(None)
    elif datatype == 'PRCP':
        tmax_values.append(None)
        tmin_values.append(None)
        tavg_values.append(None)
        prcp_values.append(value)
        snow_values.append(None)
        snwd_values.append(None)
        awnd_values.append(None)
    elif datatype == 'SNOW':
        tmax_values.append(None)
        tmin_values.append(None)
        tavg_values.append(None)
        prcp_values.append(None)
        snow_values.append(value)
        snwd_values.append(None)
        awnd_values.append(None)
    elif datatype == 'SNWD':
        tmax_values.append(None)
        tmin_values.append(None)
        tavg_values.append(None)
        prcp_values.append(None)
        snow_values.append(None)
        snwd_values.append(value)
        awnd_values.append(None)
    elif datatype == 'AWND':
        tmax_values.append(None)
        tmin_values.append(None)
        tavg_values.append(None)
        prcp_values.append(None)
        snow_values.append(None)
        snwd_values.append(None)
        awnd_values.append(value)

    dates.append(date)
    stations.append(station)

# Create a new DataFrame
new_data = {
    'date': dates,
    'station': stations,
    'TMAX': tmax_values,
    'TMIN': tmin_values,
    'TAVG': tavg_values,
    'PRCP': prcp_values,
    'SNOW': snow_values,
    'SNWD': snwd_values,
    'AWND': awnd_values
}

new_df = pd.DataFrame(new_data)

# Group by date and station and keep non-null values
grouped_df = new_df.groupby(['date', 'station']).first().reset_index()

grouped_df = grouped_df.dropna(subset=['TMAX', 'TMIN', 'TAVG','PRCP','SNOW','SNWD','AWND']) #'SNOW','SNWD',

grouped_df['date'] = pd.to_datetime(grouped_df['date'], format='%Y%m%d')

# Write the grouped DataFrame to a CSV file
csv_filename = 'Outputs/grouped_df.csv'
grouped_df.to_csv(csv_filename, index=False)

# Print the first few rows of the filtered DataFrame
grouped_df.head()

,date,station,TMAX,TMIN,TAVG,PRCP,SNOW,SNWD,AWND
6856,2018-01-01,FMW00040308,274.0,251.0,266.0,361.0,0.0,0.0,28.0
6858,2018-01-01,FMW00040505,303.0,267.0,281.0,33.0,0.0,0.0,33.0
7574,2018-01-01,GQW00041415,306.0,261.0,275.0,3.0,0.0,0.0,51.0
24719,2018-01-01,USC00244558,-105.0,-138.0,-132.0,0.0,0.0,380.0,7.0
29864,2018-01-01,USW00003017,-21.0,-160.0,-113.0,0.0,0.0,0.0,24.0


In [50]:
grouped_df.shape

(71871, 9)

In [51]:
stations_df = pd.read_csv('Outputs/full_station_list.csv')
stations_df.head()

,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude
0,139.0,1948-01-01,2014-01-01,31.57020,"ABBEVILLE, AL US",0.8813,COOP:010008,METERS,-85.24820
1,239.6,1938-01-01,2015-11-01,34.21096,"ADDISON, AL US",0.5059,COOP:010063,METERS,-87.17838
2,302.1,1940-05-01,1962-03-01,34.41667,"ADDISON CENTRAL TOWER, AL US",0.9658,COOP:010071,METERS,-87.31667
3,172.5,1995-04-01,2015-11-01,33.17835,"ALABASTER SHELBY CO AIRPORT ASOS, AL US",0.8064,COOP:010116,METERS,-86.78178
4,183.8,1949-01-01,1949-12-01,34.68910,"BELLE MINA 2 N, AL US",1.0000,COOP:010117,METERS,-86.88190


In [52]:
# Split 'id' column into two columns
stations_df[['station_code', 'station_name']] = stations_df['id'].str.split(':', expand=True)

# Drop the original 'id' column if needed
stations_df = stations_df.drop(columns=['id'])

In [53]:
stations_df.head()

,elevation,mindate,maxdate,latitude,name,datacoverage,elevationUnit,longitude,station_code,station_name
0,139.0,1948-01-01,2014-01-01,31.57020,"ABBEVILLE, AL US",0.8813,METERS,-85.24820,COOP,010008
1,239.6,1938-01-01,2015-11-01,34.21096,"ADDISON, AL US",0.5059,METERS,-87.17838,COOP,010063
2,302.1,1940-05-01,1962-03-01,34.41667,"ADDISON CENTRAL TOWER, AL US",0.9658,METERS,-87.31667,COOP,010071
3,172.5,1995-04-01,2015-11-01,33.17835,"ALABASTER SHELBY CO AIRPORT ASOS, AL US",0.8064,METERS,-86.78178,COOP,010116
4,183.8,1949-01-01,1949-12-01,34.68910,"BELLE MINA 2 N, AL US",1.0000,METERS,-86.88190,COOP,010117


In [54]:
# Load the stations.csv file into stations_df
# stations_df = pd.read_csv('full_station_list.csv')
grouped_df= pd.read_csv('Outputs/grouped_df.csv')

# Merge the two DataFrames based on 'station' using a left join
grouped_df_detailed = pd.merge(grouped_df, stations_df, 
                               left_on='station', right_on='station_name', how='left')

# Drop the redundant columns (from stations_df)
grouped_df_detailed.drop(columns=['station_name'], inplace=True)

grouped_df_detailed=grouped_df_detailed[['station','name','latitude','longitude','elevation','date','TAVG','TMAX','TMIN','PRCP','SNOW','SNWD','AWND']]

# grouped_df_detailed['date'] = pd.to_datetime(grouped_df_detailed['date'], format='%Y%m%d')

grouped_df_detailed[['station_name', 'state']] = grouped_df_detailed['name'].str.split(',', n=1, expand=True)

# Filter out rows where state doesn't end with 'US'
grouped_df_detailed = grouped_df_detailed[grouped_df_detailed['state'].str.strip().str.endswith('US')]

# Write the grouped DataFrame to a CSV file
csv_filename = 'Outputs/readings.csv'
grouped_df_detailed.to_csv(csv_filename, index=False)

grouped_df_detailed.head()

,station,name,latitude,longitude,elevation,date,TAVG,TMAX,TMIN,PRCP,SNOW,SNWD,AWND,station_name,state
2,GQW00041415,"GUAM INTERNATIONAL AIRPORT, US",13.48333,144.80000,77.4,2018-01-01,275.0,306.0,261.0,3.0,0.0,0.0,51.0,GUAM INTERNATIONAL AIRPORT,US
3,USC00244558,"KALISPELL GLACIER AIRPORT, MT US",48.30408,-114.26426,903.2,2018-01-01,-132.0,-105.0,-138.0,0.0,0.0,380.0,7.0,KALISPELL GLACIER AIRPORT,MT US
4,USW00003017,"DENVER INTERNATIONAL AIRPORT, CO US",39.84657,-104.65623,1647.2,2018-01-01,-113.0,-21.0,-160.0,0.0,0.0,0.0,24.0,DENVER INTERNATIONAL AIRPORT,CO US
5,USW00003812,"ASHEVILLE REGIONAL AIRPORT, NC US",35.43178,-82.53787,645.6,2018-01-01,-80.0,-60.0,-110.0,0.0,0.0,0.0,68.0,ASHEVILLE REGIONAL AIRPORT,NC US
6,USW00003816,"PADUCAH BARKLEY REGIONAL AIRPORT, KY US",37.05635,-88.77425,124.1,2018-01-01,-123.0,-82.0,-160.0,0.0,0.0,0.0,40.0,PADUCAH BARKLEY REGIONAL AIRPORT,KY US


In [55]:
grouped_df_detailed.shape

(71141, 15)

In [56]:
wildfire=pd.read_csv('Resources/data.csv')

wildfires=wildfire[['LATITUDE', 'LONGITUDE', 'FIRE_SIZE','NWCG_REPORTING_UNIT_NAME', 'FIRE_SIZE_CLASS', 'FIRE_YEAR',
       'FPA_ID', 'FIRE_CODE', 'NWCG_CAUSE_CLASSIFICATION',
       'NWCG_GENERAL_CAUSE', 'FIRE_NAME', 'DISCOVERY_DATE', 'CONT_DATE',
       'DISCOVERY_TIME', 'CONT_TIME', 'STATE', 'COUNTY', 'FIPS_CODE'
       ]]
wildfires=wildfires[
    (wildfires['FIRE_YEAR'].isin([2018]))
]
wildfires.head()

,LATITUDE,LONGITUDE,FIRE_SIZE,NWCG_REPORTING_UNIT_NAME,FIRE_SIZE_CLASS,FIRE_YEAR,FPA_ID,FIRE_CODE,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,FIRE_NAME,DISCOVERY_DATE,CONT_DATE,DISCOVERY_TIME,CONT_TIME,STATE,COUNTY,FIPS_CODE
2045714,46.275833,-114.379167,0.1,Bitterroot National Forest,A,2018,FS-6911076,EKS4,Natural,Natural,BLODGETT,8/22/2018,8/22/2018,1625.0,1740.0,MT,081,30081.0
2045715,46.404167,-113.921944,0.1,Bitterroot National Forest,A,2018,FS-6908885,L1RX,Human,Equipment and vehicle use,CORLEY GULCH,7/26/2018,7/28/2018,1225.0,1653.0,MT,081,30081.0
2045716,46.245833,-114.308889,1.0,Bitterroot National Forest,B,2018,FS-6898061,L49X,Human,Recreation and ceremony,CANYON CREEK,9/21/2018,9/23/2018,1305.0,1241.0,MT,081,30081.0
2045717,45.784722,-114.033056,0.1,Bitterroot National Forest,A,2018,FS-6890683,EKS4,Natural,Natural,MAYNARD CREEK,8/17/2018,NaN,1723.0,NaN,MT,081,30081.0
2045718,45.986944,-113.807222,0.1,Bitterroot National Forest,A,2018,FS-6888073,EKS4,Natural,Natural,BLUE,8/12/2018,8/12/2018,1031.0,1334.0,MT,081,30081.0


In [57]:
grouped_df_detailed=pd.read_csv('Outputs/readings.csv')

In [58]:
grouped_df_detailed.columns

Index(['station', 'name', 'latitude', 'longitude', 'elevation', 'date', 'TAVG',
       'TMAX', 'TMIN', 'PRCP', 'SNOW', 'SNWD', 'AWND', 'station_name',
       'state'],
      dtype='object')

In [59]:
# Load or generate your data, and any other necessary preprocessing steps.

# Convert dates to datetime objects
wildfires['DISCOVERY_DATE'] = pd.to_datetime(wildfires['DISCOVERY_DATE'])
wildfires['CONT_DATE'] = pd.to_datetime(wildfires['CONT_DATE'])
grouped_df_detailed['date'] = pd.to_datetime(grouped_df_detailed['date'])

# Create a KD-Tree for efficient nearest neighbor search
tree_data = grouped_df_detailed[['latitude', 'longitude']].values
tree = cKDTree(tree_data)

def find_nearest_match(row, tree, df):
    _, nearest_idx = tree.query([row['LATITUDE'], row['LONGITUDE']])
    nearest_row = df.iloc[nearest_idx]
    filtered_nearest_row = nearest_row[nearest_row['date'] == row['DISCOVERY_DATE']]
    return filtered_nearest_row

# Create a list to hold the merged rows
merged_rows = []

# Iterate over each row in the filtered wildfires DataFrame
for idx, row in wildfires.iterrows():
    nearest_row = find_nearest_match(row, tree, grouped_df_detailed)
    merged_row = pd.concat([row, nearest_row], axis=0)  # Concatenate rows vertically
    merged_rows.append(merged_row)

# Concatenate the list of merged rows into a DataFrame
merged_results = pd.concat(merged_rows, axis=1).T

# Save the resulting DataFrame to a CSV file
csv_filename = 'Outputs/merged_results_2019.csv'
merged_results.to_csv(csv_filename, index=False)

merged_results.head(10)

KeyError: False

In [ ]:
#Takes 40-45 minutes to run
warnings.filterwarnings("ignore")

wildfires['DISCOVERY_DATE'] = pd.to_datetime(wildfires['DISCOVERY_DATE'])
wildfires['CONT_DATE'] = pd.to_datetime(wildfires['CONT_DATE'])
grouped_df_detailed['date'] = pd.to_datetime(grouped_df_detailed['date'])

def find_nearest_match(row, df, date_col, lat_col, lon_col):
    date_diff = abs((df[date_col] - row['DISCOVERY_DATE']).dt.days)
    lat_diff = abs(df[lat_col] - row['LATITUDE'])
    lon_diff = abs(df[lon_col] - row['LONGITUDE'])
    total_diff = date_diff + lat_diff + lon_diff
    nearest_idx = total_diff.idxmin()
    return df.loc[nearest_idx]

# Create an empty list to hold the merged rows
merged_rows = []

# Iterate over each row in the filtered wildfires DataFrame
for idx, row in wildfires.iterrows():
    nearest_row = find_nearest_match(row, grouped_df_detailed, 'date', 'latitude', 'longitude')
    merged_row = pd.concat([row, nearest_row])
    merged_rows.append(merged_row)

# Concatenate the list of merged rows into a DataFrame
merged_results = pd.concat(merged_rows, axis=1).T

merged_results=merged_results[['LATITUDE', 'LONGITUDE','COUNTY', 'FIPS_CODE','FIRE_SIZE', 'FIRE_SIZE_CLASS',
       'NWCG_CAUSE_CLASSIFICATION', 'NWCG_GENERAL_CAUSE', 'FIRE_NAME',
       'DISCOVERY_DATE', 'CONT_DATE', 'DISCOVERY_TIME', 'CONT_TIME', 'STATE',
       'station', 'station_name','state', 'latitude', 'longitude', 'elevation', 'date', 'TAVG',
       'TMAX', 'TMIN','PRCP','SNOW','SNWD','AWND']]

csv_filename = 'Outputs/merged_results.csv'
merged_results.to_csv(csv_filename, index=False)

merged_results.head(10)


,LATITUDE,LONGITUDE,COUNTY,FIPS_CODE,FIRE_SIZE,FIRE_SIZE_CLASS,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,FIRE_NAME,DISCOVERY_DATE,...,longitude,elevation,date,TAVG,TMAX,TMIN,PRCP,SNOW,SNWD,AWND
0,46.275833,-114.379167,081,30081.0,0.1,A,Natural,Natural,BLODGETT,2018-08-22,...,-114.09376,973.8,2018-08-22,150.0,267.0,61.0,0.0,0.0,0.0,22.0
1,46.404167,-113.921944,081,30081.0,0.1,A,Human,Equipment and vehicle use,CORLEY GULCH,2018-07-26,...,-114.09376,973.8,2018-07-26,231.0,333.0,122.0,0.0,0.0,0.0,24.0
2,46.245833,-114.308889,081,30081.0,1.0,B,Human,Recreation and ceremony,CANYON CREEK,2018-09-21,...,-114.09376,973.8,2018-09-21,126.0,250.0,44.0,0.0,0.0,0.0,11.0
3,45.784722,-114.033056,081,30081.0,0.1,A,Natural,Natural,MAYNARD CREEK,2018-08-17,...,-114.09376,973.8,2018-08-17,215.0,311.0,111.0,0.0,0.0,0.0,26.0
4,45.986944,-113.807222,081,30081.0,0.1,A,Natural,Natural,BLUE,2018-08-12,...,-114.09376,973.8,2018-08-12,252.0,289.0,156.0,0.0,0.0,0.0,55.0
5,46.023056,-113.799722,081,30081.0,0.1,A,Natural,Natural,POLLYWOG,2018-08-11,...,-114.09376,973.8,2018-08-11,245.0,378.0,117.0,0.0,0.0,0.0,26.0
6,45.913611,-114.6675,049,16049.0,0.1,A,Natural,Natural,CEDAR,2018-07-25,...,-114.09376,973.8,2018-07-25,231.0,333.0,117.0,0.0,0.0,0.0,19.0
7,46.121111,-114.239167,081,30081.0,0.1,A,Natural,Natural,DOUBLE STRIKE,2018-06-08,...,-114.09376,973.8,2018-06-08,169.0,278.0,89.0,0.0,0.0,0.0,13.0
8,45.868333,-113.804167,081,30081.0,0.25,A,Natural,Natural,MEADOW,2018-08-17,...,-114.09376,973.8,2018-08-17,215.0,311.0,111.0,0.0,0.0,0.0,26.0
9,45.914444,-114.635278,049,16049.0,0.1,A,Natural,Natural,MT GEORGE 2,2018-07-24,...,-114.09376,973.8,2018-07-24,214.0,339.0,89.0,0.0,0.0,0.0,21.0


In [ ]:
# Suppress warnings
warnings.filterwarnings("ignore")

# Read the CSV file
merged_results = pd.read_csv('Outputs/merged_results_2018.csv')

# Select specific columns from the DataFrame
us_data = merged_results[['FIRE_NAME', 'STATE', 'FIPS_CODE', 'LATITUDE', 'LONGITUDE', 'DISCOVERY_DATE', 'CONT_DATE', 'station_name','state', 'latitude', 'longitude', 'date',
                                    'NWCG_CAUSE_CLASSIFICATION', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
                                    'elevation', 'TAVG', 'TMAX', 'TMIN', 'PRCP','SNOW','SNWD','AWND']]

# Rename the columns
us_data.columns = ['FIRE_NAME', 'STATE', 'FIPS_CODE', 'FIRE_LATITUDE', 'FIRE_LONGITUDE', 'FIRE_DATE', 'CONTAIN_DATE', 'CLOSEST_STATION', 'STATION_STATE', 
                             'STATION_LAT', 'STATION_LON', 'READINGS_DATE',
                             'CAUSE_CLASSIFICATION', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
                             'ELEVATION', 'TAVG', 'TMAX', 'TMIN', 'PRCP','SNOW','SNWD','AWND']

# Convert 'CONTAIN_DATE' and 'FIRE_DATE' columns to datetime
us_data['CONTAIN_DATE'] = pd.to_datetime(us_data['CONTAIN_DATE'])
us_data['FIRE_DATE'] = pd.to_datetime(us_data['FIRE_DATE'])

# Calculate the difference between 'CONTAIN_DATE' and 'FIRE_DATE'
us_data['DAYS_TO_CONTAIN'] = (us_data['CONTAIN_DATE'] - us_data_2018_test['FIRE_DATE']).dt.days

# Replace any NaN values in 'DAYS_TO_CONTAIN' with 1
us_data['DAYS_TO_CONTAIN'].fillna(0, inplace=True)

# Define float and int columns
float_columns = {
    'FIRE_LATITUDE': float,
    'FIRE_LONGITUDE': float,
    'STATION_LAT': float,
    'STATION_LON': float,
    'FIRE_SIZE': float,
    'ELEVATION': float,
    'TAVG': float,
    'TMAX': float,
    'TMIN': float,
    'PRCP': float
}

int_columns = {
    'DAYS_TO_CONTAIN': int,
}

# Convert columns to the specified data types
us_data = us_data.astype({**float_columns, **int_columns})

# Save the DataFrame to a CSV file
csv_filename = 'Outputs/Annual_data/us_data_2017.csv'
us_data.to_csv(csv_filename, index=False)

# Display the last 20 rows of the DataFrame
us_data.tail(10)


,FIRE_NAME,STATE,FIPS_CODE,FIRE_LATITUDE,FIRE_LONGITUDE,FIRE_DATE,CONTAIN_DATE,CLOSEST_STATION,STATION_STATE,STATION_LAT,...,FIRE_SIZE_CLASS,ELEVATION,TAVG,TMAX,TMIN,PRCP,SNOW,SNWD,AWND,DAYS_TO_CONTAIN
80850,325942,AZ,4013.0,33.299521,-111.813080,2018-09-02,2018-09-02,TUCSON INTERNATIONAL AIRPORT,AZ US,32.13153,...,A,777.6,298.0,339.0,200.0,81.0,0.0,0.0,39.0,0
80851,53892,AZ,4013.0,33.204753,-111.678537,2018-07-07,2018-07-07,TUCSON INTERNATIONAL AIRPORT,AZ US,32.13153,...,A,777.6,358.0,417.0,294.0,0.0,0.0,0.0,39.0,0
80852,249704,AZ,4013.0,33.851074,-112.148687,2018-07-07,2018-07-07,TUCSON INTERNATIONAL AIRPORT,AZ US,32.13153,...,A,777.6,358.0,417.0,294.0,0.0,0.0,0.0,39.0,0
80853,188245,AZ,4027.0,32.698503,-114.603127,2018-07-07,2018-07-07,MCCARRAN INTERNATIONAL AIRPORT,NV US,36.07190,...,A,662.8,386.0,428.0,339.0,0.0,0.0,0.0,31.0,0
80854,4809,AZ,4013.0,33.424078,-112.252009,2018-01-06,2018-01-06,TUCSON INTERNATIONAL AIRPORT,AZ US,32.13153,...,A,777.6,154.0,250.0,83.0,0.0,0.0,0.0,31.0,0
80855,TURNPIKE RD/BALAZS RD,CT,9013.0,41.885375,-72.261468,2018-11-17,2018-11-17,HARTFORD BRADLEY INTERNATIONAL AIRPORT,CT US,41.93742,...,A,51.6,38.0,72.0,0.0,0.0,0.0,130.0,38.0,0
80856,21 PLYMOUTH ROAD,CT,9013.0,41.985374,-72.446195,2018-12-12,2018-12-12,HARTFORD BRADLEY INTERNATIONAL AIRPORT,CT US,41.93742,...,A,51.6,-11.0,39.0,-66.0,0.0,0.0,0.0,34.0,0
80857,257 STAGECOACH ROAD,CT,9007.0,41.459430,-72.662758,2018-04-16,2018-04-16,HARTFORD BRADLEY INTERNATIONAL AIRPORT,CT US,41.93742,...,A,51.6,21.0,67.0,-5.0,546.0,15.0,30.0,45.0,0
80858,KINGS HWY AND GOOSE HILL RD,CT,9007.0,41.435494,-72.359669,2018-04-23,2018-04-23,HARTFORD BRADLEY INTERNATIONAL AIRPORT,CT US,41.93742,...,A,51.6,98.0,200.0,-10.0,0.0,0.0,0.0,29.0,0
80859,36 GREEN LANE,CT,9007.0,41.481765,-72.681206,2018-11-09,2018-11-09,HARTFORD BRADLEY INTERNATIONAL AIRPORT,CT US,41.93742,...,A,51.6,29.0,72.0,-27.0,183.0,0.0,0.0,30.0,0
